In [3]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# 1단계: 필요한 데이터 다운로드
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

# 2단계: 데이터 탐색
from nltk.corpus import movie_reviews

# 데이터셋 크기 파악
print(f"전체 영화 리뷰 수: {len(movie_reviews.fileids())}")
print(f"카테고리: {movie_reviews.categories()}")  # ['neg', 'pos']
print(f"부정 리뷰: {len(movie_reviews.fileids(categories='neg'))}개")
print(f"긍정 리뷰: {len(movie_reviews.fileids(categories='pos'))}개")

# 3단계: 첫 번째 리뷰 살펴보기
first_review_id = movie_reviews.fileids()[0]
first_review = movie_reviews.raw(first_review_id)
print(f"\n첫 번째 리뷰 ID: {first_review_id}")
print(f"원문 일부:\n{first_review[:200]}")

# 4단계: 토큰화 결과 확인
sentences = movie_reviews.sents(first_review_id)  # 문장 단위 토큰화
words = movie_reviews.words(first_review_id)      # 단어 단위 토큰화

print(f"\n문장 토큰화 (첫 2개):")
for i, sent in enumerate(sentences[:2]):
    print(f"  {i+1}: {sent}")

print(f"\n단어 토큰화 (첫 20개): {words[:20]}")

전체 영화 리뷰 수: 2000
카테고리: ['neg', 'pos']
부정 리뷰: 1000개
긍정 리뷰: 1000개

첫 번째 리뷰 ID: neg/cv000_29416.txt
원문 일부:
plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
w

문장 토큰화 (첫 2개):
  1: ['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.']
  2: ['they', 'get', 'into', 'an', 'accident', '.']

단어 토큰화 (첫 20개): ['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an']


[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\sally\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sally\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# RegexpTokenizer: 정규표현식으로 정확한 토큰화
# stopwords : 문법적 기능을 제거하고 단어에 집중
# 상위 N개 단어 선택 : 메모리효율성과 노이즈 제거의 균형

In [6]:
# BOW - 수동으로 벡터 생성
# 1단계: 모든 문서를 단어 리스트로 변환
documents = [list(movie_reviews.words(fileid)) 
             for fileid in movie_reviews.fileids()]

print(f"전체 문서 수: {len(documents)}")
print(f"첫 문서의 단어 수: {len(documents[0])}")
print(f"첫 문서의 첫 50개 단어:\n{documents[0][:50]}")

# 2단계: 전체 단어 빈도 계산 (불용어 제외 전)
word_count = {}
for doc in documents:
    for word in doc:
        word_count[word] = word_count.get(word, 0) + 1

# 상위 10개 빈도 단어 확인
sorted_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print("\n상위 10개 빈도 단어:")
for i, (word, count) in enumerate(sorted_words[:10], 1):
    print(f"  {i}. '{word}': {count}회")

# 3단계: 불용어 제거 후 처리
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# 정규표현식으로 3글자 이상의 단어만 추출
tokenizer = RegexpTokenizer(r"[\w']{3,}")
# 영어 불용어 로드
english_stops = set(stopwords.words('english'))

# 모든 리뷰를 토큰화하고 불용어 제거
processed_documents = []
for fileid in movie_reviews.fileids():
    raw_text = movie_reviews.raw(fileid)
    tokens = [token for token in tokenizer.tokenize(raw_text) 
              if token not in english_stops]
    processed_documents.append(tokens)

# 처리 후 단어 빈도 재계산
word_count_processed = {}
for doc in processed_documents:
    for word in doc:
        word_count_processed[word] = word_count_processed.get(word, 0) + 1

sorted_processed = sorted(word_count_processed.items(), 
                         key=lambda x: x[1], reverse=True)

print(f"\n전체 서로 다른 단어 수: {len(sorted_processed)}")
print("\n처리 후 상위 10개 단어:")
for i, (word, count) in enumerate(sorted_processed[:10], 1):
    print(f"  {i}. '{word}': {count}회")

# 4단계: 특성 선택 (상위 1000개 단어)
word_features = [word for word, count in sorted_processed[:1000]]
print(f"\n특성으로 선택된 단어 수: {len(word_features)}")
print(f"특성 예시: {word_features[:20]}")

전체 문서 수: 2000
첫 문서의 단어 수: 879
첫 문서의 첫 50개 단어:
['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'", 's', 'the', 'deal', '?', 'watch']

상위 10개 빈도 단어:
  1. ',': 77717회
  2. 'the': 76529회
  3. '.': 65876회
  4. 'a': 38106회
  5. 'and': 35576회
  6. 'of': 34123회
  7. 'to': 31937회
  8. ''': 30585회
  9. 'is': 25195회
  10. 'in': 21822회

전체 서로 다른 단어 수: 43011

처리 후 상위 10개 단어:
  1. 'film': 8935회
  2. 'one': 5791회
  3. 'movie': 5538회
  4. 'like': 3690회
  5. 'even': 2564회
  6. 'time': 2409회
  7. 'good': 2407회
  8. 'story': 2136회
  9. 'would': 2084회
  10. 'much': 2049회

특성으로 선택된 단어 수: 1000
특성 예시: ['film', 'one', 'movie', 'like', 'even', 'time', 'good', 'story', 'would', 'much', 'also', 'get', 'character', 'two', 'well', '

In [7]:
# processed_documents[0]  # 문장을 토큰화(3개의 연속된 문장, 불용어제거)
for doc in processed_documents[:5]:
    print(len(doc), end=" ")

323 130 262 278 391 

In [8]:
processed_documents[0][:3]

['plot', 'two', 'teen']

In [9]:
# 각 문서의 고정된 길이의 벡터로 변환(모든 문서가 같은 차원 )
# 기계학습 알고리즘의 입력 형식으로 변환
def document_features(document, word_features):
    """
    문서를 특성 벡터로 변환
    
    Args:
        document: 토큰화된 단어 리스트
        word_features: 특성으로 사용할 단어 리스트
    
    Returns:
        document의 각 특성에 대한 빈도 리스트
    """
    # 문서 내 단어 빈도 계산
    word_count = {}
    for word in document:
        word_count[word] = word_count.get(word, 0) + 1
    
    # 특성 벡터 생성
    features = []
    for word in word_features:
        # 특성 단어가 문서에 없으면 0
        features.append(word_count.get(word, 0))
    
    return features

# 테스트 실행
test_features = ['one', 'two', 'teen', 'couples', 'solo']
test_doc = ['two', 'two', 'couples']
result = document_features(test_doc, test_features)

print("테스트 단어 리스트:", test_features)
print("테스트 문서:", test_doc)
print("결과 벡터:", result)
print("→ 'two'가 2번, 'couples'가 1번, 나머지는 0")

# 모든 문서에 대해 특성 벡터 생성
feature_sets = [document_features(doc, word_features) 
                 for doc in processed_documents]

print(f"\n생성된 특성 벡터 수: {len(feature_sets)}")
print(f"각 벡터의 차원: {len(feature_sets[0])}")
print(f"\n첫 문서 벡터 (처음 20개):")
for i, (word, count) in enumerate(zip(word_features[:20], feature_sets[0][:20])):
    print(f"  '{word}': {count}")

테스트 단어 리스트: ['one', 'two', 'teen', 'couples', 'solo']
테스트 문서: ['two', 'two', 'couples']
결과 벡터: [0, 2, 0, 1, 0]
→ 'two'가 2번, 'couples'가 1번, 나머지는 0

생성된 특성 벡터 수: 2000
각 벡터의 차원: 1000

첫 문서 벡터 (처음 20개):
  'film': 5
  'one': 3
  'movie': 6
  'like': 3
  'even': 3
  'time': 0
  'good': 2
  'story': 0
  'would': 1
  'much': 0
  'also': 1
  'get': 3
  'character': 1
  'two': 2
  'well': 1
  'first': 0
  'characters': 1
  'see': 2
  'way': 3
  'make': 5


In [10]:
word_features[:3],  feature_sets[0][:3],  len(word_features), len(feature_sets[0])

(['film', 'one', 'movie'], [5, 3, 6], 1000, 1000)

In [11]:
# 1 데이터 준비
reivews = [  movie_reviews.raw(fileid) for fileid in movie_reviews.fileids() ]
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(vocabulary=word_features)
reviews_cv = cv.fit_transform(reivews)
print(f'특성수 : {cv.get_feature_names_out()[:5]}')
reviews_cv.shape

특성수 : ['film' 'one' 'movie' 'like' 'even']


(2000, 1000)

In [1]:
# https://drive.google.com/file/d/1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o/view?usp=sharing
import gdown

In [12]:
# https://drive.google.com/file/d/1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o/view?usp=sharing

import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)

In [13]:
df.head(2)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워


In [ ]:
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
okt = Okt()
def custom_tokenizer(doc):
    """
    형태소 분석 후 명사, 동사, 형용사만 추출
    """
    pos_tags = okt.pos(doc)
    tokens = [word for word, pos in pos_tags 
              if pos in ['Noun', 'Verb', 'Adjective']]
    return tokens

daum_cv = CountVectorizer(
    max_features=1000,
    tokenizer=custom_tokenizer
)

reviews = df.review
daum_dtm  = daum_cv.fit_transform(reviews)
original_review = reviews[0]  # 첫 번째 리뷰
print(f"원본 리뷰 (처음 200자):\n{original_review[:200]}\n")

# 문서의 뒤 절반을 query로 사용 (부분 검색 시나리오)
midpoint = len(original_review) // 2
query_text = original_review[midpoint:]  # 뒤 절반
print(f"쿼리 텍스트 (처음 150자):\n{query_text[:150]}\n")

# 2단계: 쿼리 문서를 벡터로 변환
query_vector = daum_cv.transform([query_text])
print(f"쿼리 벡터 크기: {query_vector.shape}")

# 유사도 분석
scores = cosine_similarity(query_vector,daum_dtm)
most_simular_idx = np.argmax(scores)
# scores[most_simular_idx], reviews[most_simular_idx]

In [27]:
scores[0].argsort()[::-1][:5]

array([12500, 11750, 12517,  3023,     0])

In [28]:
df.shape

(14725, 4)

In [29]:
query_text

'만 보는 내내 하품만'

In [30]:
np.array(reviews)[scores[0].argsort()[::-1][:5]]

array(['보는 내내 눈시울이 ㅠㅠ', '여전한 군바리 국가지배...보는 내내 슬펐다.', '보는 내내 너무 괴로웠다',
       '보는 내내 설레였다', '돈 들인건 티가 나지만 보는 내내 하품만'], dtype=object)

In [31]:
# 개선 TF-IDF
# 단어의 상대적 중요도를 반영한 벡터화 기법
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
# TF란 특정 단어가 문서에서 얼마나 자주 나타나는지 비율
# 해당 단어의 빈도 / 문서의 전체 단어 수 
    # '좋다' 문서에서 10번 -> 해당 문서는 100단어 -> 10/100
# IDF(Inverse Document Frequency) : 단어가 전체 문장에서 얼마나 희귀한지, 드문지
    # IDF는 가중치를 낮추는 것
# 전체 문서 / 해당 단어 포함 문서
# log (전체 문서 / 해당 단어 포함 문서) 단어의 가중치를 낮추기 위해서 log 적용
# 2000개 문서 중 100만 '좋다' log(2000/100) = 2.99
# TF-IDF = TF x IDF --> 특정 문서에서 의미있는 단어에 높은 가중치 부여

In [34]:
tfidf_cv = TfidfVectorizer(
    max_features=1000,
    tokenizer=custom_tokenizer
)
tfid_dtm = tfidf_cv.fit_transform(reviews)
tfid_dtm.shape

c:\Users\sally\SKNPYWORKSPACE\streamlit\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(14725, 1000)

In [35]:
# 쿼리 벡터화
count_cv = CountVectorizer(
    max_features=1000,
    tokenizer=custom_tokenizer
)
count_dtm = count_cv.fit_transform(reviews)
tfidf_cv = TfidfVectorizer(
    max_features=1000,
    tokenizer=custom_tokenizer
)
tfid_dtm = tfidf_cv.fit_transform(reviews)
count_dtm.shape, tfid_dtm.shape

c:\Users\sally\SKNPYWORKSPACE\streamlit\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


((14725, 1000), (14725, 1000))

In [36]:
# 해당되는 문서에 대해서 쿼리 벡터화
query_count = count_cv.transform([query_text])
query_tfidf = tfidf_cv.transform([query_text])
# 코사인 유사도 계산
count_sim = cosine_similarity(query_count,count_dtm)[0]
tfid_sim = cosine_similarity(query_count,tfid_dtm)[0]

In [38]:
query_text

'만 보는 내내 하품만'

In [ ]:
a = np.array([1,2,3,4,5])
a.argsort()     # 오름차순으로 인덱스
(-a).argsort()  # 값의 마이너스는 내림차순

array([4, 3, 2, 1, 0])

In [46]:
top5_count_index = (-count_sim).argsort()[:5]
top5_tfidf_index = (-tfid_sim).argsort()[:5]

In [47]:
reviews[top5_count_index]

3023                    보는 내내 설레였다
11750    여전한 군바리 국가지배...보는 내내 슬펐다.
12500                보는 내내 눈시울이 ㅠㅠ
12517                보는 내내 너무 괴로웠다
0           돈 들인건 티가 나지만 보는 내내 하품만
Name: review, dtype: object

In [49]:
reviews[top5_tfidf_index][1:]

11750    여전한 군바리 국가지배...보는 내내 슬펐다.
12517                보는 내내 너무 괴로웠다
12500                보는 내내 눈시울이 ㅠㅠ
12172         영화 보는 내내  분노했고 미안했다.
Name: review, dtype: object

In [ ]:
my_review = '숙면을 하기 좋은 영화, 강추'
my_review count 방식이나 또는 tf-idf 방식으로 벡터화 한 후 .. 전체 리뷰ㅣ를 